In [ ]:
#| hide
from timescale_vector import client 

# Timescale-vector

> Python library for storing vector data in Postgres

This file will become your README and also the index of your documentation.

## Install

```sh
pip install timescale_vector
```

## How to use

Load up your postgres credentials. Safest way is with a .env file:

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

In [ ]:
_ = load_dotenv(find_dotenv()) 
connection_string  = os.environ['PG_CONNECTION_STRING'] 

Next, create the client. 

This takes three arguments: 
- A connection string
- The name of the collection
- Number of dimensions

In [ ]:
#| hide
import asyncpg

In [ ]:
#| hide
con = await asyncpg.connect(connection_string)
await con.execute("DROP TABLE IF EXISTS my_data;")
await con.close()

In [ ]:
vec  = client.Async(connection_string, "my_data", 2)

Next, create the tables for the collection:

In [ ]:
await vec.create_tables()

Next, insert some data. The data record contains:
- A uuid to uniquely identify the emedding
- A json blob of metadata about the embedding
- The text the embedding represents
- The embedding itself

Because this data already includes uuids we only allow upserts

In [ ]:
import uuid

In [ ]:
await vec.upsert([\
    (uuid.uuid4(), '''{"animal":"fox"}''', "the brown fox", [1.0,1.3]),\
    (uuid.uuid4(), '''{"animal":"fox", "action":"jump"}''', "jumped over the", [1.0,10.8]),\
])

Now you can query for similar items:

In [ ]:
await vec.search([1.0, 9.0])

[<Record id=UUID('25df4eea-a17f-42c2-9426-d09b8ca40e32') metadata='{"action": "jump", "animal": "fox"}' contents='jumped over the' embedding=array([ 1. , 10.8], dtype=float32) ?column?=0.00016793422934946456>,
 <Record id=UUID('605e3ff5-3503-4006-826f-c84ecbb535d4') metadata='{"animal": "fox"}' contents='the brown fox' embedding=array([1. , 1.3], dtype=float32) ?column?=0.14489260377438218>]

You can specify the number of records to return.

In [ ]:
await vec.search([1.0, 9.0], k=1)

[<Record id=UUID('25df4eea-a17f-42c2-9426-d09b8ca40e32') metadata='{"action": "jump", "animal": "fox"}' contents='jumped over the' embedding=array([ 1. , 10.8], dtype=float32) ?column?=0.00016793422934946456>]

You can also specify a filter on the metadata as a simple dictionary

In [ ]:
await vec.search([1.0, 9.0], k=1, filter={"action": "jump"})

[<Record id=UUID('25df4eea-a17f-42c2-9426-d09b8ca40e32') metadata='{"action": "jump", "animal": "fox"}' contents='jumped over the' embedding=array([ 1. , 10.8], dtype=float32) ?column?=0.00016793422934946456>]